<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

This notebook should be run with **Default Spark 3.3 & Python 3.9** or **Python 3.10** runtime environment. 

If you are viewing this in Watson Studio and do not see Python 3.9.x in the upper right corner of your screen, please update the runtime now.

It requires service credentials for the following services:
  * Watson OpenScale
  * Watson Machine Learning 
  * DB2
  
The notebook will train, create and deploy a German Credit Risk model, and configure OpenScale to monitor that deployment. Model Evaluation will also be triggered and published Fact will be retrieved.

**Note**: The AI Factsheets add-on must be installed on the CPD cluster for the facts to be published and retrieved successfully.

### Contents

- [Setup](#setup)
- [Model Building and Deployment](#model)
- [Configure OpenScale](#openscale)
- [Monitor Configurations](#monitor)
- [MRM Evaluation](#mrm)

# 1. Setup <a name="setup"></a>

## 1.1 Package installation

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Install WML and WOS SDKs

!pip install --upgrade ibm-watson-machine-learning | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

In [ ]:
# Install pyspark if runtime environment doesn't include Spark

!pip install --upgrade pyspark==3.3.0 --no-cache | tail -n 1

**Note** - Restart the kernel now to use the updated libraries.

## 1.2 Configure credentials

Provide OpenScale, Watson Machine Learning, and DB2 service credentials.

In [ ]:
cpd_url = "***"
cpd_username = "***"
cpd_password = "***"

cpd_url = cpd_url.rstrip("/")

WOS_CREDENTIALS = {
    "url": cpd_url,
    "username": cpd_username,
    "password": cpd_password
}

WML_CREDENTIALS = {
    "url": "***",
    "password": "***",
    "username": "***",
    "instance_id": "***",
    "version": "4.7"
}

DB2_CREDENTIALS = {
    "hostname": "***",
    "username": "***",
    "password": "***",
    "database_name": "***",
    "port": 50000,
    "ssl": False
}

# Location details of German Credit Risk training data
TRAINING_DATA_SCHEMA_NAME = "***"
TRAINING_DATA_TABLE_NAME = "***"

# Location details of German Credit Risk evaluation data 
EVALUATION_DATA_SCHEMA_NAME = "***"
EVALUATION_DATA_TABLE_NAME = TEST_DATA_SET_NAME = "***"

# 2. Model Building and Deployment <a name="model"></a>

In this section you will learn how to train Spark MLLib model and next deploy it as web-service using Watson Machine Learning service.

## 2.1 Load the training data from Github <a name="model"></a>

In [ ]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_data_biased_training.csv

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import json

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
df_data = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)
df_data.head()

## 2.2 Explore data

In [ ]:
df_data.printSchema()

In [ ]:
print("Number of records: " + str(df_data.count()))

## 2.3 Create a model

In [ ]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

spark_df.printSchema()

The code below creates a **Random Forest Classifier** with Spark, setting up string indexers for the categorical features and the label column. Finally, this notebook creates a pipeline including the indexers and the model and does an initial **Area Under ROC** evaluation of the model.

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model
from pyspark.ml.feature import SQLTransformer

features = [x for x in spark_df.columns if x != 'Risk']
categorical_features = ['CheckingStatus', 'CreditHistory', 'LoanPurpose', 'ExistingSavings', 'EmploymentDuration', 'Sex', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans', 'Housing', 'Job', 'Telephone', 'ForeignWorker']
categorical_num_features = [x + '_IX' for x in categorical_features]
si_list = [StringIndexer(inputCol=x, outputCol=y) for x, y in zip(categorical_features, categorical_num_features)]
va_features = VectorAssembler(inputCols=categorical_num_features + [x for x in features if x not in categorical_features], outputCol="features")
si_label = StringIndexer(inputCol="Risk", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_label.labels)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

classifier = RandomForestClassifier(featuresCol="features")
pipeline = Pipeline(stages= si_list + [si_label, va_features, classifier, label_converter])

model = pipeline.fit(train_data)

In [ ]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction",  metricName='areaUnderROC')
area_under_curve = evaluatorDT.evaluate(predictions)

evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction",  metricName='areaUnderPR')
area_under_PR = evaluatorDT.evaluate(predictions)

# Default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve, "areaUnderPR = %g" % area_under_PR)

## 2.4 Create and Save the Model

Save and deploy the German Credit Risk model into the WML instance that is designated as **Pre-Production**.

In [ ]:
PRE_PROD_MODEL_NAME = "GCR Model"
PRE_PROD_DEPLOYMENT_NAME = "GCR Model Deployment"
SPACE_ID = "***"

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
print(wml_client.version)
wml_client.set.default_space(SPACE_ID)

### 2.4.1 Cleaning up existing model, deployments, subscriptions

In [ ]:
from ibm_watson_machine_learning.wml_client_error import WMLClientError

deployments_list = wml_client.deployments.get_details()
models_to_delete = []
deployments_deleted = []
for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    dep_model_name = wml_client.repository.get_details(model_id)["metadata"]["name"]
    deployment_id = deployment["metadata"]["id"]
    if deployment["metadata"]["name"] == PRE_PROD_DEPLOYMENT_NAME or dep_model_name == PRE_PROD_MODEL_NAME:
        deployments_deleted.append(deployment_id)
        if model_id not in models_to_delete:
            models_to_delete.append(model_id)

for deployment_id in deployments_deleted:
    try:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
    except WMLClientError as wce:
        if "deployment_does_not_exist" in wce.error_msg:
            # Shadow deployment
            pass
        else:
            raise wce

for model_id in models_to_delete:
    print("Deleting model id", model_id)
    wml_client.repository.delete(model_id)
wml_client.repository.list_models()

### 2.4.2 Save the Model

In [ ]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: PRE_PROD_MODEL_NAME,
    wml_client.repository.ModelMetaNames.TYPE: 'mllib_3.3',
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: wml_client.software_specifications.get_id_by_name("spark-mllib_3.3")
}

published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props, 
                                                        training_data=train_data, pipeline=pipeline)
model_uid = wml_client.repository.get_model_id(published_model_details)
print("Model UID:" + model_uid)

## 2.5 Deploy the Model

The next section of the notebook deploys the model as a RESTful web service in Watson Machine Learning. The deployed model will have a scoring URL you can use to send data to the model for predictions.

In [ ]:
deployment_details = wml_client.deployments.create(
    model_uid, 
    meta_props={
        wml_client.deployments.ConfigurationMetaNames.NAME: "{}".format(PRE_PROD_DEPLOYMENT_NAME),
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)

scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
deployment_uid = wml_client.deployments.get_uid(deployment_details)

print("Scoring URL: {}".format(scoring_url))
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

### 2.5.1 Sample scoring

In [ ]:
fields = ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings",
                  "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration",
                  "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents",
                  "Telephone", "ForeignWorker"]
values = [
            ["no_checking", 13, "credits_paid_to_date", "car_new", 1343, "100_to_500", "1_to_4", 2, "female", "none", 3,
             "savings_insurance", 46, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["no_checking", 24, "prior_payments_delayed", "furniture", 4567, "500_to_1000", "1_to_4", 4, "male", "none",
             4, "savings_insurance", 36, "none", "free", 2, "management_self-employed", 1, "none", "yes"],
        ]

scoring_payload = {"input_data": [{"fields": fields, "values": values}]}

In [ ]:
scoring_response = wml_client.deployments.score(deployment_uid, scoring_payload)
scoring_response

# 3. Configure OpenScale <a name="openscale"></a>

The subsequent cells will now import the necessary libraries and set up a Python OpenScale client.

In [ ]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import APIClient

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

### 3.1 Initialize the APIClient

In [ ]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import APIClient as WOSAPIClient

SERVICE_INSTANCE_ID = DATA_MART_ID = "00000000-0000-0000-0000-000000000000"
wos_client = WOSAPIClient(
    authenticator=CloudPakForDataAuthenticator(
        url=cpd_url,
        username=cpd_username,
        password=cpd_password,
        disable_ssl_verification=True
    ),
    service_url=cpd_url,
    service_instance_id=SERVICE_INSTANCE_ID
)

print(wos_client.version)

In [ ]:
# Listing service providers

wos_client.service_providers.show()

In [ ]:
# Copy the ID of your service provider from the `id` column in the output of the cell above

SERVICE_PROVIDER_ID = "***"

### 3.2 Add Subscription

In [ ]:
# Remove existing credit risk subscription

wos_client.subscriptions.show()

In [ ]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == model_uid:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print('Deleted existing subscription for model', sub_model_id)

In [ ]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=DATA_MART_ID, 
    service_provider_id=SERVICE_PROVIDER_ID, deployment_id = deployment_uid, deployment_space_id=SPACE_ID).result['resources'][0]
asset_deployment_details

In [ ]:
model_asset_details_from_deployment=wos_client.service_providers.get_deployment_asset(data_mart_id=DATA_MART_ID,
    service_provider_id=SERVICE_PROVIDER_ID, deployment_id=deployment_uid, deployment_space_id=SPACE_ID)
model_asset_details_from_deployment

In [ ]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=DATA_MART_ID,
        background_mode=False,
        service_provider_id=SERVICE_PROVIDER_ID,
        asset=Asset(
            asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
            name=model_asset_details_from_deployment["entity"]["asset"]["name"],
            url=model_asset_details_from_deployment["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        ),
        deployment=AssetDeploymentRequest(
            deployment_id=asset_deployment_details['metadata']['guid'],
            name=asset_deployment_details['entity']['name'],
            deployment_type= DeploymentTypes.ONLINE,
            url=asset_deployment_details['entity']['scoring_endpoint']['url']
        ),
        asset_properties=AssetPropertiesRequest(
            label_column='Risk',
            probability_fields=['probability'],
            prediction_field='predictedLabel',
            feature_fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
            categorical_fields = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"],
            training_data_reference=TrainingDataReference(
            type="db2",
            location=DB2TrainingDataReferenceLocation(
                    table_name=TRAINING_DATA_TABLE_NAME,
                    schema_name=TRAINING_DATA_SCHEMA_NAME
                ),
                connection=DB2TrainingDataReferenceConnection.from_dict(DB2_CREDENTIALS)
            ),
            training_data_schema=SparkStruct.from_dict(model_asset_details_from_deployment["entity"]["asset_properties"]["training_data_schema"])
        )
    ).result

subscription_id = subscription_details.metadata.id
subscription_id

In [ ]:
# Check Payload Logging table status

import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(
    type=DataSetTypes.PAYLOAD_LOGGING, 
    target_target_id=subscription_id, 
    target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id

if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

In [ ]:
# List all Subscription Data Sets

wos_client.data_sets.show()

### 3.3 Score the model before Monitor Configurations

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model.

In [ ]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
payload = {
    wml_client.deployments.ScoringMetaNames.INPUT_DATA: [payload_scoring]
}
scoring_response = wml_client.deployments.score(deployment_uid, payload)

print('Single record scoring result:', '\n fields:', scoring_response['predictions'][0]['fields'], '\n values: ', scoring_response['predictions'][0]['values'][0])

In [ ]:
# Check whether WML payload logging worked; else manually store payload records

import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    print("Payload logging did not happen, performing explicit payload logging.")
    wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response={"fields": scoring_response['predictions'][0]['fields'], "values":scoring_response['predictions'][0]['values']},
                   response_time=460
               )])
    time.sleep(5)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

# 4. Monitor Configurations <a name="monitor"></a>

### 4.1 Quality Monitor

The cell below waits ten seconds to allow the payload logging table to be set up before it begins enabling monitors. First, it turns on the quality (accuracy) monitor and sets an alert threshold of 80%.

The second paramater supplied, `min_feedback_data_size`, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement and `max_rows_per_evaluation` specifies the maximum number of the records for which quality metrics can be evaluated. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback logging endpoint.

In [ ]:
import time

time.sleep(10)
max_records = None
#Update the max_records value when you want to consider it during quality metrics evaluation
#max_records = 80

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters =  dict()
parameters["min_feedback_data_size"] = 50
if max_records is not None:
    parameters["max_rows_per_evaluation"] = max_records

thresholds = [
    {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
    }
]

quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATA_MART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

### 4.2 Fairness Monitor

The code below configures fairness monitoring for our model. It turns on monitoring for two features, `Sex` and `Age`. In each case, we must specify:
  * Which model feature to monitor.
  * One or more **majority** groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes.
  * One or more **minority** groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes.
  * The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 95%).

Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score.

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 50
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATA_MART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result

fairness_monitor_instance_id =fairness_monitor_details.metadata.id
fairness_monitor_instance_id

### 4.3 Drift Monitor

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_samples": 50,
    "drift_threshold": 0.1,
    "train_drift_model": True,
    "enable_model_drift": True,
    "enable_data_drift": True
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=DATA_MART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
drift_monitor_instance_id

### 4.4 Explainability

In [ ]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=DATA_MART_ID,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id

In [ ]:
# Run Sample Explanation

pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]

print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types, subscription_id=subscription_id).result

print(result)

### 4.5 MRM Monitor

In [ ]:
# Configuring MRM

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
mrm_details = wos_client.monitor_instances.create(
    data_mart_id=DATA_MART_ID,
    background_mode=False,
    monitor_definition_id="mrm",
    target=target,
    parameters=parameters
).result

MRM_MONITOR_INSTANCE_ID = mrm_details.metadata.id
MRM_MONITOR_INSTANCE_ID

In [ ]:
wos_client.monitor_definitions.show()

## 5. MRM Evaluation <a name="mrm"></a>

In [ ]:
import datetime
import requests
import json

time.sleep(20)

payload = json.dumps({
  "type": "db2",
  "connection": DB2_CREDENTIALS,
  "location": {
    "schema_name": EVALUATION_DATA_SCHEMA_NAME,
    "table_name": EVALUATION_DATA_TABLE_NAME
    }
})

file_name = "test_data.json"

with open(file_name, 'w') as file:
    file.write(payload)

response = wos_client.monitor_instances.mrm.evaluate_risk(
                monitor_instance_id = MRM_MONITOR_INSTANCE_ID,
                test_data_set_name = TEST_DATA_SET_NAME,
                feedback_data_path = file_name,
                content_type = "application/json")

print(response.result._to_dict())

### 5.1 Checking MRM evaluation progress

In [ ]:
import time

def get_risk_evaluations():
    
    response = wos_client.monitor_instances.mrm.get_risk_evaluation(
                    monitor_instance_id = MRM_MONITOR_INSTANCE_ID).result
    
    return response._to_dict()

risk_evaluations_resp = get_risk_evaluations()

while risk_evaluations_resp["entity"]["status"]["state"] == "UPLOAD_IN_PROGRESS":
    print("UPLOAD_IN_PROGRESS")
    time.sleep(50)
    risk_evaluations_resp = get_risk_evaluations()

while risk_evaluations_resp["entity"]["status"]["state"] not in ["finished", "error"]:
    print("EVALUATION_IN_PROGRESS")
    time.sleep(50)
    risk_evaluations_resp = get_risk_evaluations()

mrm_evaluation_state = risk_evaluations_resp["entity"]["status"]["state"]
if mrm_evaluation_state == "finished":
    print("EVALUATION_COMPLETED")
else:
    print("MRM evaluation failed with state {}, error {}".format(mrm_evaluation_state, risk_evaluations_resp.json()["entity"]["status"]))

### 5.2 Get Published Fact

In [ ]:
def get_published_fact():

    url = "{}/v1/aigov/model_inventory/models/{}/system_facts?space_id={}&deployment_id={}".format(
        cpd_url, model_uid, SPACE_ID, deployment_uid)
    
    headers = {
        'Authorization': 'Bearer {}'.format(wos_client.authenticator.token_manager.get_token()),
        'Accept': 'application/json'
    }

    response = requests.request("GET", url, headers=headers, verify=False)
    return response

# Wait for a minute before fetching the published Fact
time.sleep(60)

published_fact = get_published_fact()
print(json.dumps(published_fact.json(), indent=4))

Congratulations! You have reached the end of the demo notebook. Thanks for trying it out :)

### Authors
Developed by [Harshit Sharma](mailto:harshit2@in.ibm.com), Staff Software Engineer, Watson OpenScale